In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("./juliaEAM.jl")
include("../../src/lammpsIO.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using Unitful: Å, nm
using StaticArrays: SVector
using Molly
using LinearAlgebra
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using ProgressMeter
using Random

  Activating project at `~/Documents/ABCD_J/EAM/lammps_wrap`


## Define Aluminum

In [2]:
include("./juliaEAM.jl")

al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

## `AtomCalculators` force/energy calculator and `Molly` simulator
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
    f_atomstress::Any  # Holds the atomic level stresses function
    pair_coeff_string::String  # Holds the pair coefficient string
end

include("./simulator.jl")


simulate! (generic function with 1 method)

In [3]:
# eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces,calculate_atomstress,"pair_coeff * * Al99.eam.alloy Al")
# function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
#     n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 
    
#     fs = interaction.f_forces(interaction.calculator, sys, neighbors_all)

#     # Add penalty term to forces
#     if penalty_coords != nothing
#         fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
#         # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
#     end
#     return fs
# end

eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces_LAMMPS,calculate_atomstress,"pair_coeff * * Al99.eam.alloy Al")
function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
    n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 
    
    fs = interaction.f_forces(sys, interaction.pair_coeff_string)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
        # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
    end
    return fs
end

In [ ]:
function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly, attypes = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)

    DNF = DistanceNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
        n_steps=1e3,
        dist_cutoff=8u"Å")
    TNF = TreeNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)), 
        n_steps=1e3,
        dist_cutoff=8u"Å")

    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al", atom_type=string(attypes[ia])) for (ia,a) in enumerate(molly_atoms_init)],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DNF,
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system_dump (generic function with 1 method)

## Initialize sys

In [5]:
filename_dump = "./out_vacancy_big_bcc.dump"
molly_system = initialize_system_dump(filename_dump = filename_dump)
neighbors_all = get_neighbors_all(molly_system)

16368-element Vector{Vector{Int64}}:
 [257, 2193, 2449, 3095, 3097, 3098, 3099, 3100, 3101, 3102  …  12309, 12725, 12726, 12728, 12757, 12758, 12759, 12760, 12789, 13269]
 [3, 4, 5, 6, 7, 8, 9, 12, 33, 34  …  15411, 15889, 15890, 15891, 15892, 15921, 15922, 15923, 15924, 15955]
 [2, 4, 5, 6, 7, 8, 9, 12, 33, 34  …  9208, 9238, 9718, 12277, 12533, 14469, 15890, 15891, 15892, 15923]
 [2, 3, 5, 6, 7, 8, 33, 34, 35, 36  …  15889, 15890, 15891, 15892, 15919, 15921, 15922, 15923, 15924, 15955]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  8247, 8694, 8695, 8726, 9206, 12533, 14469, 15890, 15891, 15923]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  8160, 8161, 8162, 8163, 8164, 8165, 8166, 8167, 8168, 8172]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  8160, 8161, 8162, 8163, 8164, 8165, 8166, 8167, 8168, 8172]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  8246, 8247, 8694, 8726, 8727, 8758, 8759, 9206, 15891, 15923]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  8163, 8164, 8165, 8166, 8167, 8168, 8169, 8170, 8171, 8172]
 [5, 6, 7

energy minimization

In [6]:
simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=20, step_size_minimize=5e-3u"ps", tol=1e-6u"eV/Å")
Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
         n_threads=1, frozen_atoms=[], neig_interval=5, print_nsteps=true,
         mass=26.9815u"u")
neighbors_all = get_neighbors_all(molly_system)

16368-element Vector{Vector{Int64}}:
 [257, 2193, 2449, 3095, 3097, 3098, 3099, 3100, 3101, 3102  …  12309, 12725, 12726, 12728, 12757, 12758, 12759, 12760, 12789, 13269]
 [3, 4, 5, 6, 7, 8, 9, 12, 33, 34  …  15411, 15889, 15890, 15891, 15892, 15921, 15922, 15923, 15924, 15955]
 [2, 4, 5, 6, 7, 8, 9, 12, 33, 34  …  9208, 9238, 9718, 12277, 12533, 14469, 15890, 15891, 15892, 15923]
 [2, 3, 5, 6, 7, 8, 33, 34, 35, 36  …  15889, 15890, 15891, 15892, 15919, 15921, 15922, 15923, 15924, 15955]
 [2, 3, 4, 6, 7, 8, 9, 10, 11, 12  …  8247, 8694, 8695, 8726, 9206, 12533, 14469, 15890, 15891, 15923]
 [2, 3, 4, 5, 7, 8, 9, 10, 11, 12  …  8160, 8161, 8162, 8163, 8164, 8165, 8166, 8167, 8168, 8172]
 [2, 3, 4, 5, 6, 8, 9, 10, 11, 12  …  8160, 8161, 8162, 8163, 8164, 8165, 8166, 8167, 8168, 8172]
 [2, 3, 4, 5, 6, 7, 9, 10, 11, 12  …  8246, 8247, 8694, 8726, 8727, 8758, 8759, 9206, 15891, 15923]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  8163, 8164, 8165, 8166, 8167, 8168, 8169, 8170, 8171, 8172]
 [5, 6, 7

run ABC

In [7]:
frozen_atoms = []
nopenalty_atoms = []
N_free = length(molly_system.coords)-length(nopenalty_atoms)
print("exclude ",length(nopenalty_atoms)," atoms from E_phi calculation\n")
print("using ",N_free," atoms for E_phi calculation\n")

# sigma = sqrt(0.006*3*N_free)
sigma = sqrt(0.25)
W = 0.12
@printf("sigma^2 = %e, %e Å/dof^1/2\n W = %e eV\n",ustrip(sigma^2), ustrip(sigma/sqrt(3*N_free)),ustrip(W))

simulator = ABCSimulator(sigma=sigma*u" Å", W=W*u"eV", 
                         max_steps=1000, max_steps_minimize=30, step_size_minimize=5e-3u"ps", tol=1e-3u"eV/Å")

simulate!(molly_system, simulator, eamJulia, n_threads=1, 
         #   fname="output_stress_cube8.txt", fname_dump="stress_cube8.dump", fname_min_dump="min_stress_cube8.dump",
         fname="test.txt", fname_dump="test.dump", fname_min_dump="test.dump", # for speed test
         neig_interval=30, loggers_interval=10, dump_interval=10, start_dump=0,
         minimize_only=false, 
         d_boost=1e-6u"Å", 
         frozen_atoms=frozen_atoms, nopenalty_atoms=nopenalty_atoms, 
         p_drop = 1-1/32, p_keep=0, n_memory=0, n_search=100,
         p_stress = 1-12/16384)

exclude 0 atoms from E_phi calculation
using 16368 atoms for E_phi calculation
sigma^2 = 2.500000e-01, 2.256377e-03 Å/dof^1/2
 W = 1.200000e-01 eV
step 0: -54985.68180225951 eV


Progress: 100%|█████████████████████████████████████████| Time: 2:38:55


System with 16368 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[64.78996211866995 Å, 64.78996211866995 Å, 64.78996211866995 Å])